# Lesson 5: Leveraging Assistants API for SQL Databases

In [ ]:
import helper
import json
import os
import time

from IPython.display import clear_output
from helper import get_positive_cases_for_state_on_date
from helper import get_hospitalized_increase_for_state_on_date
from openai import AzureOpenAI

## Launch the Assistant API

In [ ]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# I) Create assistant
assistant = client.beta.assistants.create(
  instructions="""You are an assistant answering questions 
                  about a Covid dataset.""",
  model="gpt-4-1106", 
  tools=helper.tools_sql)

# II) Create thread
thread = client.beta.threads.create()
print(thread)

In [ ]:
# III) Add message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska
               the 2021-03-05?"""
)
print(message)

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

In [ ]:
# IV) Run assistant on thread

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

## Leverage the function calling with Assistants API